In [1]:
#collecting data
import numpy as np
import scipy.io
mat = scipy.io.loadmat('dataset_connected_NYC.mat') #while tryin in terminal, start path with Documents

## Preprocessing data

In [2]:
friendship_new = mat['friendship_new']
friendship_old = mat['friendship_old']
selected_checkins = mat['selected_checkins']
selected_users_IDs = mat['selected_users_IDs']
temp = mat['selected_venue_IDs']
selected_venue_IDs = []
for i in range(len(temp)):
    temper = temp[i][0]
    selected_venue_IDs.append(temper[0])
#print(len(selected_venue_IDs)) #sanity check
#print(len(temp)) #should be same as above
print(selected_checkins)

[[   1801      39   17807      70]
 [    105      39    5708      33]
 [    571      39    3968     300]
 ...
 [    718      59   12596      70]
 [   1446      59  104201     279]
 [   2305      60 1692889     190]]


In [3]:
# 1. rebuild node index
offset1 = max(selected_checkins[:,0]);
dumy, dumy, n = np.unique(selected_checkins[:,1],return_index=True,return_inverse=True, axis=0)
selected_checkins[:,1] = n+offset1;
#print(selected_checkins[:,1])
#print(offset1)
#print(n)
offset2 = max(selected_checkins[:,1]);
dumy, dumy, n = np.unique(selected_checkins[:,2],return_index=True,return_inverse=True, axis=0)
selected_checkins[:,2] = n+offset2;
#print(selected_checkins[:,2])
#print(offset2)
#print(n)
offset3 = max(selected_checkins[:,2]);
dumy, dumy, n = np.unique(selected_checkins[:,3],return_index=True,return_inverse=True, axis=0)
selected_checkins[:,3] = n+offset3;
#print(selected_checkins[:,3])
#print(offset3)
#print(n)

num_node_total = max(map(max, selected_checkins)) #max of the entire matrix = 8117
#print(min(map(min, selected_checkins)))

In [4]:
# 2. prepare checkins per user (fast)
#user_chechkins is a cell in MATLAB; using Python List for the same
user_checkins = [[] for _ in range(selected_users_IDs.shape[0])]
user_checkins_counter=np.zeros((len(selected_users_IDs),1)).astype(np.int64)
ind = selected_checkins[:,0].argsort(axis=0)
temp_checkins = np.array([selected_checkins[i,:] for i in ind]) #will not exactly be similar to MATLAB op
u,m,n = np.unique(temp_checkins[:,0],return_index=True,return_inverse=True, axis=0)
m=m.reshape(-1,1)
u=np.array(u)
counters = np.vstack((m[1:],temp_checkins.shape[0]))-m #+1 is not there because indexing starts from 0
ini_val=0
for i in range(0,u.shape[0]):
    user_checkins[u[i]-1].append(temp_checkins[ini_val:ini_val+counters[i,0],:])
    user_checkins[u[i]-1]=np.array(user_checkins[u[i]-1])[0,:,:].T
    user_checkins[u[i]-1].astype(np.int64)
    ini_val=ini_val+counters[i,0]
    user_checkins_counter[u[i]-1]=counters[i].astype(np.int64)
user_checkins[4023].shape #for debugging

#print(np.count_nonzero(user_checkins_counter)) #for debugging
#len(user_checkins_counter)
tttt = user_checkins[0]
print(tttt[:,0])

[   1 4061 7418 7881]


In [5]:
# 3. random walk
from scipy.sparse import csr_matrix
num_node = len(selected_users_IDs)
network=csr_matrix((np.ones((len(friendship_old),)), (friendship_old[:,0]-1, friendship_old[:,1]-1)), shape=(num_node, num_node))
network=network+network.T

node_list=[[] for _ in range(num_node)]
node_list_len = np.zeros((num_node,));
num_walk = 10;
len_walk = 80;
(indx,indy) = network.T.nonzero()
#for i in range(len(indx)):
#    indx[i]+=1
#    indy[i]+=1

temp, m, n = np.unique(indx,return_index=True,return_inverse=True, axis=0) #check m for debugging
print(temp)
print(m)
print(n)
m=np.array(m).reshape(-1,1)
node_list_len=np.vstack((m[1:],len(indx))) - m #+1 is not there because indexing starts from 0
ini_val=0
temp=np.array(temp)
node_list_len=node_list_len[:,0] #for indexing

for i in range(0,temp.shape[0]):
    node_list[temp[i]].append(indy[ini_val:ini_val+node_list_len[i]])
    ini_val=ini_val+node_list_len[i]
#node_list #for debugging match with MATLAB
#node_list_len.shape = np.count_nonzero(node_list_len) has to be as written in MATLAB script


[   0    1    2 ... 4021 4022 4023]
[    0    16    25 ... 17441 17442 17444]
[   0    0    0 ... 4022 4022 4023]


In [6]:
# let's have a walk over social network (friendship)
import random
walks = np.zeros((num_walk*num_node,len_walk),dtype = np.int64);
for ww in range(num_walk):
    for ii in range(num_node):
        seq = np.zeros((len_walk,),dtype=int)
        seq[0] = ii
        current_e = ii
        for jj in range(len_walk-1):
            rand_ind = random.randint(0, node_list_len[seq[jj]]-1);
            tempvar = node_list[seq[jj]][0] 
            #print(node_list_len[seq[jj]])
            #print(rand_ind)
            #print(tempvar.shape)
            seq[jj+1] = tempvar[rand_ind];
        
        walks[ii+(ww-1)*num_node,:] = seq
        
#preprocessing for removing 0 quantities
for i in range(walks.shape[0]):
    for j in range(walks.shape[1]):
        walks[i,j] = walks[i,j] +1
#print(min(map(min, walks))) #should be 1 and 4024
print(walks)

[[   1 1279    1 ... 3886   69  708]
 [   2    1 1497 ...  729 1362  729]
 [   3  401   43 ... 2050 1509 3117]
 ...
 [4022 1990  495 ...  626 1168 1740]
 [4023 2430 2192 ... 3742 3881 3591]
 [4024 2823 4024 ...  539 3282  389]]


In [7]:
# 4. prepare negative sample table in advance (fast)
# social relationship
from collections import Counter
(dumy,r) = network.nonzero(); #MATLAB gives [y,x]; Python gives (x,y)
temptab = Counter(r) #len(temptab) = size(tab_degree,1) #Counter gives unique elements in r
tab2= [[] for _ in range(len(temptab))]

for i in range(len(temptab)):
    tab2[i].append(temptab[i])
tab2=np.array(tab2)[:,0] #when printing temptab, not in asc. order but here it becomes asc. wrt keys

tab1=np.unique(r)
tab3=[[] for _ in range(len(tab1))]
tot=sum(tab2)
for i in range(len(tab1)):
    tab3[i]=round((tab2[i]*100)/tot,4)
tab_degree=np.stack((tab1,tab2,tab3),axis=1) # equivalent of tab_degree = tabulate(r) in MATLAB

freq = np.array([round(np.power(i,0.75),4) for i in tab_degree[:,2]] )
den=float(sum(freq))
neg_sam_table_social = np.repeat((1+tab_degree[:,0]),np.around(1000000*freq/sum(freq)).astype(np.int64))
neg_sam_table_social=neg_sam_table_social.astype(np.int64) # unigram with 0.75 power
#neg_sam_table_social[740] check for debugging
del temptab,tab1,tab2,tab3,tab_degree,freq,den
max(neg_sam_table_social)

4024

In [8]:
#checkins: user, venue, time, semantic, with node type normalization for each node domain

neg_sam_table_mobility_norm = [[] for _ in range(4)]
for ii in range(len(neg_sam_table_mobility_norm)):
    tab1=np.array([elem for elem in range(max(temp_checkins[:,ii]))])#tab1 has all elements;even with frequency 0
    temptab = Counter(temp_checkins[:,ii]) #correct
    tab2= np.zeros((len(tab1),))
    for i in tab1:
        tab2[i]= temptab[i+1] #because in python, index starts from 0
    tab2 = np.array(tab2)
    tot=np.sum(tab2)
    tab3=np.zeros((len(tab1),))
    for i in range(len(tab1)):
        tab3[i]= np.round((tab2[i]*100)/tot,4) if (tot) else 0
    tab3=np.array(tab3)
    tab_degree=np.stack((tab1,tab2,tab3),axis=1)
    freq = np.array([np.round(np.power(i,0.75),4) for i in tab_degree[:,2]])
    den=float(sum(freq))
    ingoes = np.repeat((tab_degree[:,0]+1),np.around(100000*freq/sum(freq)).astype(np.int64))
    neg_sam_table_mobility_norm[ii].append(ingoes)

    del tab1,tab2,i,tab3,tab_degree,freq,ingoes,tot

neg_sam_table_mobility_norm = [np.array(x).astype(np.int64) for x in neg_sam_table_mobility_norm]

#neg_sam_table_mobility1 = neg_sam_table_mobility_norm[3][0]
#neg_sam_table_mobility1[5]
print(neg_sam_table_mobility_norm)

[array([[   1,    1,    1, ..., 4024, 4024, 4024]]), array([[4024, 4024, 4024, ..., 4191, 4191, 4191]]), array([[4191, 4191, 4191, ..., 7818, 7818, 7818]]), array([[7818, 7818, 7818, ..., 8117, 8117, 8117]])]


In [9]:
# LBSN2vec
dim_emb = 128
num_epoch = 1
num_threads =  4
K_neg = 10
win_size = 10
learning_rate = 0.001


embs_ini = (np.random.uniform(size=(num_node_total,dim_emb))-0.5)/dim_emb 
temp = np.sum(np.power(embs_ini,2),axis=1)
embs_len = np.power(temp,0.5)
den = embs_len
for i in range(dim_emb-1):
    den=np.vstack((den,embs_len))
den=den.T
#print(den.shape)
#print(embs_ini.shape)
embs_ini = np.divide(embs_ini,den)

mobility_ratio = 0.2

In [10]:
import learn_LBSN2Vec_embedding

embs = learn_LBSN2Vec_embedding.driver_fn(walks.T,user_checkins, user_checkins_counter,embs_ini.T,learning_rate, 
                                K_neg,neg_sam_table_social, win_size, neg_sam_table_mobility_norm,
                                num_epoch, mobility_ratio);

this is num_w 40240
this is num_wl 80
[[   1    2    3 ... 4022 4023 4024]
 [1279    1  401 ... 1990 2430 2823]
 [   1 1497   43 ...  495 2192 4024]
 ...
 [3886  729 2050 ...  626 3742  539]
 [  69 1362 1509 ... 1168 3881 3282]
 [ 708  729 3117 ... 1740 3591  389]]


KeyboardInterrupt: 

In [16]:
neg_sam_table_mobility = neg_sam_table_mobility_norm
neg_sam_table_mobility1 = neg_sam_table_mobility[0][0]
table_size_mobility1 = neg_sam_table_mobility1.shape
table_size_mobility1[0]

100502